# Convert TF Object Detection Models for Open Vino Inference Engine

## configuration

In [1]:
import tensorflow as tf
import os

workspace_dir = '/home/manuel/Bachelor_Arbeit/TensorFlow/workspace'
dataset = 'OI_Animals_Augmented_9_3000'
model = 'ssd_mobilenet_v2_coco_2018_03_29_out'

checkpoint = 70956
model_type_json = 'ssd_api'
pipeline_config_file = 'ssd_mob'
img_size = 300
channel = 3
batch_size = 1

model_type = {
    'faster': '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/faster_rcnn_support.json',
    'faster_api': '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/faster_rcnn_support_api_v1.14.json',
    'ssd' : '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_support.json',
    'ssd_api' : '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_support_api_v1.14.json',
    'rfcn' : '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/rfcn_support.json',
    'rfcn_api' : '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/rfcn_support_api_v1.14.json'
}

config_file = {
    'ssd_mob': 'ssd_mobilenet_v2_coco.config', 
    'ssd_mob_oi': 'ssd_mobilenet_v2_oid_v4.config', 
    'ssd_inc':  'ssd_inception_v2_coco.config',
    'faster_inc': 'faster_rcnn_inception_v2_coco.config',
    'rfcn_res101':  'rfcn_resnet101_pets.config',
    'faster_res50': 'faster_rcnn_resnet50_coco.config'
}

pb_file = os.path.join(workspace_dir, dataset, model, 'exported', 'frozen_inference_graph.pb')

configuration_file = model_type[model_type_json]

pipeline = os.path.join(workspace_dir, dataset, model, config_file[pipeline_config_file])

plugin_device = 'MYRIAD'

data_type = 'FP16'

input_shape_str = '['+ str(batch_size) +',' + str(img_size) + ',' + str(img_size) + ',' + str(channel) + ']'

output_dir = os.path.join(workspace_dir, dataset, model, 'open_vino')

mo_tf_path = '/opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py'
input_shape_str

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

'[1,300,300,3]'

## export frozen graph

In [2]:
checkpoint_str = os.path.join(workspace_dir, dataset, model, 'model.ckpt-' + str(checkpoint))
exported = os.path.join(workspace_dir, dataset, model, 'exported')

!python3 /home/manuel/Bachelor_Arbeit/TensorFlow/models/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path $pipeline \
    --trained_checkpoint_prefix $checkpoint_str \
    --output_directory $exported

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

W0206 15:48:48.931965 139838770353984 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f2e83651710>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f2e83651710>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0206 15:48:49.069517 139838770353984 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f2e836de358>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: conv

W0206 15:48:49.741608 139838770353984 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f2e834e7c88>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f2e834e7c88>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0206 15:48:49.807885 139838770353984 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f2e836f0b00>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Batch

W0206 15:48:50.556386 139838770353984 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f2e8336fa20>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f2e8336fa20>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0206 15:48:50.601332 139838770353984 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f2e8361a1d0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Batch

W0206 15:48:50.990292 139838770353984 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f2e834ab438>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f2e834ab438>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0206 15:48:51.081272 139838770353984 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f2e83166d30>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach th

...

W0206 15:48:55.687769 139838770353984 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f2e81d9f4e0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f2e81d9f4e0>>: AssertionError: Bad argument number for Name: 3, expecting 4
INFO:tensorflow:depth of additional conv before box predictor: 0
I0206 15:48:55.695311 139838770353984 convolutional_box_predictor.py:151] depth of additional conv before box predictor: 0
W0206 15:48:55.763589 139838770353984 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f2e81d66860>> could not be transformed and will be executed as-is. Please report this to the AutgoGrap

W0206 15:48:56.191661 139838770353984 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f2e82739470>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f2e82739470>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0206 15:48:56.265618 139838770353984 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f2e81be9f98>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python

175 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*,.*Initializer.*,.*Regularizer.*,.*BiasAdd.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================
Incomplete shape.

Doc:
scope: The nodes in the model graph are organiz


W0206 15:48:58.469089 139838770353984 deprecation_wrapper.py:119] From /home/manuel/Bachelor_Arbeit/TensorFlow/models/object_detection/exporter.py:432: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

2020-02-06 15:48:59.428784: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-02-06 15:48:59.515832: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2699905000 Hz
2020-02-06 15:48:59.516192: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4eea6e0 executing computations on platform Host. Devices:
2020-02-06 15:48:59.516224: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0206 15:48:59.519253 139838770353984 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/te

INFO:tensorflow:SavedModel written to: /home/manuel/Bachelor_Arbeit/TensorFlow/workspace/OI_Animals_Augmented_9_3000/ssd_mobilenet_v2_coco_2018_03_29_out/exported/saved_model/saved_model.pb
I0206 15:49:05.018553 139838770353984 builder_impl.py:421] SavedModel written to: /home/manuel/Bachelor_Arbeit/TensorFlow/workspace/OI_Animals_Augmented_9_3000/ssd_mobilenet_v2_coco_2018_03_29_out/exported/saved_model/saved_model.pb

W0206 15:49:05.050184 139838770353984 deprecation_wrapper.py:119] From /home/manuel/Bachelor_Arbeit/TensorFlow/models/object_detection/utils/config_util.py:188: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.

INFO:tensorflow:Writing pipeline config file to /home/manuel/Bachelor_Arbeit/TensorFlow/workspace/OI_Animals_Augmented_9_3000/ssd_mobilenet_v2_coco_2018_03_29_out/exported/pipeline.config
I0206 15:49:05.050356 139838770353984 config_util.py:190] Writing pipeline config file to /home/manuel/Bachelor_Arbeit/TensorFlow/workspace/OI_Animals

##  convert for open vino

In [3]:

!python3 {mo_tf_path} \
--input_model {pb_file} \
--tensorflow_use_custom_operations_config {configuration_file} \
--input_shape {input_shape_str} \
--data_type {data_type} \
--input image_tensor \
--tensorflow_object_detection_api_pipeline_config {pipeline} \
--output_dir {output_dir}


Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/manuel/Bachelor_Arbeit/TensorFlow/workspace/OI_Animals_Augmented_9_3000/ssd_mobilenet_v2_coco_2018_03_29_out/exported/frozen_inference_graph.pb
	- Path for generated IR: 	/home/manuel/Bachelor_Arbeit/TensorFlow/workspace/OI_Animals_Augmented_9_3000/ssd_mobilenet_v2_coco_2018_03_29_out/open_vino
	- IR output name: 	frozen_inference_graph
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	image_tensor
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,300,300,3]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorB

## Inference

In [ ]:
from openvino.inference_engine import IENetwork, IEPlugin, IECore
import cv2
import numpy as np
import time
import os
from random import shuffle
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
workspace_dir = '/home/manuel/Bachelor_Arbeit/workspace'
dataset = 'OI_Animals_Augmented_9_2000'
model = 'ssd_inception_v2_coco_2018_01_28_out'

In [ ]:
model_xml = os.path.join(workspace_dir, dataset, model, 'open_vino', 'frozen_inference_graph.xml')
model_bin = os.path.join(workspace_dir, dataset, model, 'open_vino', 'frozen_inference_graph.bin')

labels = [l.strip() for l in open(os.path.join(workspace_dir, dataset, 'classes.txt')).readlines()]

test_images_dir = os.path.join(workspace_dir, dataset, 'validation')

assert os.path.isfile(model_bin)
assert os.path.isfile(model_xml)
assert os.path.isdir(test_images_dir)

In [ ]:
# collect images
if os.path.isdir(test_images_dir):
    images = [
        os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img[-3:] == 'jpg']
    print(str(len(images)) + ' images found')
    shuffle(images)
elif os.path.isfile(test_images_dir) and test_images_dir[-3:] == 'jpg':
    print('one image found')
    images = [test_images_dir]
else:
    print('wrong input')
    exit()
    
frame = cv2.imread(images[0])
initial_h, initial_w = frame.shape[: 2]

In [ ]:
# create the infer net
ie = IECore()
net = IENetwork(model=model_xml, weights=model_bin)

exec_net = ie.load_network(network=net, num_requests=2, device_name='MYRIAD')

img_info_input_blob = None
feed_dict = {}
for blob_name in net.inputs:
    if len(net.inputs[blob_name].shape) == 4:
        input_blob = blob_name
    elif len(net.inputs[blob_name].shape) == 2:
        img_info_input_blob = blob_name
    else:
        raise RuntimeError("Unsupported {}D input layer '{}'. Only 2D and 4D input layers are supported"
                               .format(len(net.inputs[blob_name].shape), blob_name))

assert len(net.outputs) == 1, "Demo supports only single output topologies"

out_blob = next(iter(net.outputs))
n, c, h, w = net.inputs[input_blob].shape
if img_info_input_blob:
    feed_dict[img_info_input_blob] = [h, w, 1]

cur_request_id = 0
next_request_id = 1

frame = cv2.imread(images[0])
initial_h, initial_w = frame.shape[: 2]